# [1148. Article Views I](https://leetcode.com/problems/article-views-i/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Views
<pre>
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| article_id    | int     |
| author_id     | int     |
| viewer_id     | int     |
| view_date     | date    |
+---------------+---------+
</pre>
There is no primary key (column with unique values) for this table, the table may have duplicate rows.
Each row of this table indicates that some viewer viewed an article (written by some author) on some date. 
Note that equal author_id and viewer_id indicate the same person.
 

Write a solution to find all the authors that viewed at least one of their own articles.

Return the result table sorted by id in ascending order.

The result format is in the following example.

 

Example 1:

Input: 
Views table:
<pre>
+------------+-----------+-----------+------------+
| article_id | author_id | viewer_id | view_date  |
+------------+-----------+-----------+------------+
| 1          | 3         | 5         | 2019-08-01 |
| 1          | 3         | 6         | 2019-08-02 |
| 2          | 7         | 7         | 2019-08-01 |
| 2          | 7         | 6         | 2019-08-02 |
| 4          | 7         | 1         | 2019-07-22 |
| 3          | 4         | 4         | 2019-07-21 |
| 3          | 4         | 4         | 2019-07-21 |
+------------+-----------+-----------+------------+</pre>
Output: 
<pre>
+------+
| id   |
+------+
| 4    |
| 7    |
+------+</pre>

In [1]:
# Pandas schema

import pandas as pd

data = [[1, 3, 5, '2019-08-01'], [1, 3, 6, '2019-08-02'], [2, 7, 7, '2019-08-01'], [2, 7, 6, '2019-08-02'], [4, 7, 1, '2019-07-22'], [3, 4, 4, '2019-07-21'], [3, 4, 4, '2019-07-21']]
views = pd.DataFrame(data, columns=['article_id', 'author_id', 'viewer_id', 'view_date']).astype({'article_id':'Int64', 'author_id':'Int64', 'viewer_id':'Int64', 'view_date':'datetime64[ns]'})

In [2]:
# To pyspark schema

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

views_df = spark.createDataFrame(views)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/05 10:41:27 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/05 10:41:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/05 10:41:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
views_df.printSchema()

root
 |-- article_id: long (nullable = true)
 |-- author_id: long (nullable = true)
 |-- viewer_id: long (nullable = true)
 |-- view_date: timestamp (nullable = true)


In [4]:
views_df.show(truncate=False)

+----------+---------+---------+-------------------+
|article_id|author_id|viewer_id|view_date          |
+----------+---------+---------+-------------------+
|1         |3        |5        |2019-08-01 00:00:00|
|1         |3        |6        |2019-08-02 00:00:00|
|2         |7        |7        |2019-08-01 00:00:00|
|2         |7        |6        |2019-08-02 00:00:00|
|4         |7        |1        |2019-07-22 00:00:00|
|3         |4        |4        |2019-07-21 00:00:00|
|3         |4        |4        |2019-07-21 00:00:00|
+----------+---------+---------+-------------------+


In [5]:
# In spark Dataframe
import pyspark.sql.functions as F

views_df\
    .filter(F.col('author_id')==F.col('viewer_id'))\
    .select(F.col('author_id').alias('id'))\
    .distinct()\
    .orderBy('id')\
    .show()

+---+
| id|
+---+
|  4|
|  7|
+---+


In [6]:
#In spark SQL
views_df.createOrReplaceTempView('views')

spark.sql('''
select distinct author_id as id
from views
where author_id=viewer_id
order by id;
''').show()

+---+
| id|
+---+
|  4|
|  7|
+---+


In [7]:
spark.stop()